In [2]:
import sys
import time
import json
import pprint
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\etien\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\etien\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'3.5.1'

In [3]:
patent_data = open('G01N.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

In [4]:
def get_feature_names(vectorizer):
    if hasattr(vectorizer, 'get_feature_names'):
        return vectorizer.get_feature_names()
    else:
        return vectorizer.get_feature_names_out()

In [5]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes), 'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


In [16]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(
    1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X = cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=get_feature_names(cvectorizer)).T.sort_values(by=0, ascending=False)
termdf_cv.head(25)

c:\Users\etien\anaconda3\envs\NLP\lib\site-packages\sklearn\feature_extraction\text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


,0
amino acid,24339
nucleic acid,13185
amino acid sequence,11407
light source,6943
amino acids,6170
control unit,5249
nucleotide sequence,4374
variable region,3264
mass spectrometry,3091
monoclonal antibody,2917


In [17]:
termdf_cv[termdf_cv[0] > 500]
termdf_cv.to_csv('terms.tsv', sep='\t')

In [18]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000])
displacy.render(doc, style="ent", jupyter=True)